In [1]:
from __future__ import division, print_function
%matplotlib inline
from importlib import reload  # Python 3
import utils; reload(utils)
from utils import *
from keras.layers.merge import dot, add, concatenate

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5110)
Using TensorFlow backend.


In [2]:
#path = "data/ml-latest-small/"  # from https://grouplens.org/datasets/movielens/
#path = "data/ml-20m/"
path = "datas/movielens/" #from https://github.com/ChicagoBoothML/DATA___MovieLens___20M
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)

batch_size=64
#batch_size=1

In [3]:
#20M dataset
#ratings = pd.read_csv(path+'ratings.csv')

#2M dataset
#ratings = pd.read_csv(path+'ratings01.csv')

#100k dataset
ratings = pd.read_csv(path+'ratings100k.csv')

In [4]:
#show 5 first file rows
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [5]:
#nb of lines
len(ratings)

100000

### data prep

In [6]:
#read film names
movie_names = pd.read_csv(path+'movies.csv').set_index('movieId')['title'].to_dict

#https://www.datacamp.com/community/tutorials/pandas-tutorial-dataframe-python
#https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_dict.html
#https://stackoverflow.com/questions/30060616/python-pandas-to-dict-function

In [7]:
movie_names1 = pd.read_csv(path+'movies.csv').set_index('movieId')['title']
print(movie_names1[:5])

movieId
1                      Toy Story (1995)
2                        Jumanji (1995)
3               Grumpier Old Men (1995)
4              Waiting to Exhale (1995)
5    Father of the Bride Part II (1995)
Name: title, dtype: object


In [8]:
movie_names2 = pd.read_csv(path+'movies.csv').set_index('movieId')
print(movie_names2[:5])

                                      title  \
movieId                                       
1                          Toy Story (1995)   
2                            Jumanji (1995)   
3                   Grumpier Old Men (1995)   
4                  Waiting to Exhale (1995)   
5        Father of the Bride Part II (1995)   

                                              genres  
movieId                                               
1        Adventure|Animation|Children|Comedy|Fantasy  
2                         Adventure|Children|Fantasy  
3                                     Comedy|Romance  
4                               Comedy|Drama|Romance  
5                                             Comedy  


In [9]:
usersUniq = ratings.userId.unique()
moviesUniq = ratings.movieId.unique()

print(usersUniq[:20])
print(moviesUniq[:20])

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
[  2  29  32  47  50 112 151 223 253 260 293 296 318 337 367 541 589 593 653 919]


In [10]:
# userId and movieId become ditionary elements with values ranging from 0 to max len 
userUniqID2idx = {value:index for index,value in enumerate(usersUniq)}
movieUniqID2idx = {movieID:index for index,movieID in enumerate(moviesUniq)}

In [23]:
{movieID:movieUniqID2idx[movieID] for movieID in list(movieUniqID2idx)[:10]}

{1: 227,
 2: 0,
 3: 175,
 4: 1994,
 5: 1044,
 6: 364,
 7: 420,
 8: 1965,
 9: 1677,
 10: 365}

In [12]:
ratings.movieId[:20]

0       2
1      29
2      32
3      47
4      50
5     112
6     151
7     223
8     253
9     260
10    293
11    296
12    318
13    337
14    367
15    541
16    589
17    593
18    653
19    919
Name: movieId, dtype: int64

In [13]:
#We update the movie and user ids so that they are contiguous integers, which we want when using embeddings.
ratings.movieId = ratings.movieId.apply(lambda x: movieUniqID2idx[x])
ratings.userId = ratings.userId.apply(lambda x: userUniqID2idx[x])

#https://chrisalbon.com/python/data_wrangling/pandas_apply_operations_to_dataframes/

In [14]:
ratings.movieId[:15]

0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
11    11
12    12
13    13
14    14
Name: movieId, dtype: int64

In [15]:
ratings.movieId[983]

11

In [18]:
user_min, user_max, movie_min, movie_max = (ratings.userId.min(), 
    ratings.userId.max(), ratings.movieId.min(), ratings.movieId.max())

In [19]:
n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()

In [20]:
user_min, user_max, movie_min, movie_max, n_users, n_movies

(0, 701, 0, 8226, 702, 8227)

### embeddings

In [24]:
#number of latent factors in each embedding
n_factors = 50

In [25]:
np.random.seed = 42

#https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.random.seed.html

In [29]:
#Randomly split into training and validation

#Create an array of the given shape and populate it with random samples from a uniform distribution over [0, 1).
msk = np.random.rand(len(ratings)) < 0.8
print(msk)

trn = ratings[msk]
val = ratings[~msk]

[ True  True  True ...,  True False False]


In [30]:
trn[:5]

,userId,movieId,rating,timestamp
0,0,0,3.5,1112486027
1,0,1,3.5,1112484676
2,0,2,3.5,1112484819
4,0,4,3.5,1112484580
5,0,5,3.5,1094785740


In [31]:
val[:5]

,userId,movieId,rating,timestamp
3,0,3,3.5,1112484727
7,0,7,4.0,1112485573
8,0,8,4.0,1112484940
10,0,10,4.0,1112484703
12,0,12,4.0,1112484798


### Create subset for Excel

In [32]:
#We create a crosstab of the most popular movies and most movie-addicted users 
#which we'll copy into Excel for creating a simple example. 
#This isn't necessary for any of the modeling below however.
g=ratings.groupby('userId')['rating'].count()
topUsers=g.sort_values(ascending=False)[:15]

In [35]:
topUsers

userId
155    2179
585    1431
571    1326
358    1300
207    1288
393    1212
297    1127
115    1110
631    1094
613    1042
103     998
423     918
647     904
586     873
347     786
Name: rating, dtype: int64

In [36]:
top_r = ratings.join(topUsers, rsuffix='_r', how='inner', on='userId')

In [38]:
top_r.head(15)

,userId,movieId,rating,timestamp,rating_r
11454,103,1187,1.0,944917003,998
11455,103,387,3.0,944928813,998
11456,103,630,3.0,945002688,998
11457,103,681,4.0,945003357,998
11458,103,2021,3.0,944917299,998
11459,103,1,3.0,944920275,998
11460,103,3163,3.0,944924853,998
11461,103,2,3.0,945002402,998
11462,103,1045,4.0,944928872,998
11463,103,1046,3.0,944912818,998


In [33]:
g=ratings.groupby('movieId')['rating'].count()
topMovies=g.sort_values(ascending=False)[:15]

In [34]:
topMovies

movieId
11     350
369    340
12     305
182    302
17     295
9      264
178    262
16     256
331    253
652    247
227    241
205    238
392    235
242    234
237    233
Name: rating, dtype: int64

In [39]:
top_r = top_r.join(topMovies, rsuffix='_r', how='inner', on='movieId')

In [40]:
top_r.head(15)

,userId,movieId,rating,timestamp,rating_r,rating_r
11488,103,392,2.0,945002463,998,235
13216,115,392,3.0,1132711598,1110,235
19908,155,392,5.0,1039204398,2179,235
25897,207,392,5.0,942448811,1288,235
38535,297,392,3.0,992545882,1127,235
46961,358,392,4.0,1087076743,1300,235
53882,393,392,4.0,957483687,1212,235
78348,571,392,5.0,1324264879,1326,235
81197,585,392,3.0,1112498213,1431,235
86881,613,392,3.5,1122836727,1042,235


In [41]:
pd.crosstab(top_r.userId, top_r.movieId, top_r.rating, aggfunc=np.sum)

movieId,9,11,12,16,17,178,182,205,227,237,242,331,369,392,652
userId,,,,,,,,,,,,,,,
103,3.0,3.0,3.0,NaN,3.0,NaN,NaN,2.0,NaN,2.0,NaN,1.0,NaN,2.0,5.0
115,4.5,4.5,4.5,4.0,3.0,4.5,4.0,4.5,3.0,3.0,1.0,4.0,4.0,3.0,4.0
155,NaN,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
207,4.5,5.0,4.5,2.5,4.5,3.5,4.5,3.5,4.0,5.0,4.0,3.0,1.0,5.0,4.5
297,5.0,4.0,5.0,5.0,4.0,5.0,4.0,5.0,4.0,4.0,4.0,5.0,3.0,3.0,3.0
347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,3.5,NaN,NaN,NaN,4.5
358,5.0,5.0,5.0,5.0,5.0,3.5,4.0,4.0,5.0,4.0,3.0,3.0,4.5,4.0,5.0
393,4.0,5.0,3.0,4.0,5.0,2.5,2.0,5.0,NaN,4.0,1.0,5.0,NaN,4.0,3.0
423,3.5,NaN,4.5,3.5,NaN,4.5,4.0,NaN,NaN,NaN,4.0,4.5,3.0,NaN,NaN


### Dot product
The most basic model is a dot product of a movie embedding and a user embedding. Let's see how well that works:

latent factor = explicative variables
#### calculate the latent factors for each movie AND each user, so that we can predict ratings by a dot product of 2 vectors :
latent user factors vector * latent movie factors vector

In [47]:
user_min, user_max, movie_min, movie_max, n_users, n_movies, n_factors

(0, 701, 0, 8226, 702, 8227, 50)

In [48]:
user_in = Input(shape=(1,), dtype='int64', name='user_in')
u = Embedding(input_dim=n_users, output_dim=n_factors, input_length=1, embeddings_regularizer=l2(1e-4))(user_in)

movie_in = Input(shape=(1,), dtype='int64', name='movie_in')
m = Embedding(input_dim=n_movies, output_dim=n_factors, input_length=1, embeddings_regularizer=l2(1e-4))(movie_in)

In [49]:
u

<tf.Tensor 'embedding_3/Gather:0' shape=(?, 1, 50) dtype=float32>

In [51]:
#model construction, no data yet (we love TF :D)
x = dot([u, m], axes=2)
x = Flatten()(x)
model = Model([user_in, movie_in], x)
model.compile(Adam(0.001), loss='mse')

In [52]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=batch_size, epochs=1, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 79860 samples, validate on 20140 samples
Epoch 1/1
79860/79860 [==============================] - 6s - loss: 9.7538 - val_loss: 4.1809


In [53]:
model.optimizer.lr=0.01

In [54]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=batch_size, epochs=3, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 79860 samples, validate on 20140 samples
Epoch 1/3
79860/79860 [==============================] - 6s - loss: 3.0504 - val_loss: 2.7470
Epoch 2/3
79860/79860 [==============================] - 6s - loss: 2.3511 - val_loss: 2.5390
Epoch 3/3
79860/79860 [==============================] - 6s - loss: 2.1949 - val_loss: 2.4912


In [55]:
model.optimizer.lr=0.001

In [56]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=batch_size, epochs=6, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 79860 samples, validate on 20140 samples
Epoch 1/6
79860/79860 [==============================] - 6s - loss: 2.1395 - val_loss: 2.4732
Epoch 2/6
79860/79860 [==============================] - 6s - loss: 2.1130 - val_loss: 2.4668
Epoch 3/6
79860/79860 [==============================] - 6s - loss: 2.0904 - val_loss: 2.4688
Epoch 4/6
79860/79860 [==============================] - 6s - loss: 2.0727 - val_loss: 2.4690
Epoch 5/6
79860/79860 [==============================] - 6s - loss: 2.0545 - val_loss: 2.4716
Epoch 6/6
79860/79860 [==============================] - 6s - loss: 2.0359 - val_loss: 2.4769


In [57]:
#best result at epoch 2 !

### add Bias
The problem is likely to be that we don't have bias terms - that is, a single bias for each user and each movie representing how positive or negative each user is, and how good each movie is. We can add that easily by simply creating an embedding with one output for each movie and each user, and adding it to our output.